In [7]:
import numpy as np

# Constantes del sistema
G = 1.0  # Constante de gravitación universal (reescalada)
c = 1.496e11  # Unidad de distancia (m), distancia Tierra-Sol
Ms = 1.0  # Masa del Sol (reescalada)
dt = 0.2  # Paso temporal (reescalado)
num_steps = 1000 # Número de pasos de simulación

# Datos iniciales de los planetas (posición y velocidad inicial reescaladas)
# Formato: [x, y, vx, vy, masa]
"""planetas = {
    "Sol": [0.0 * 2.5, 0.0, 0.0, 0.0, 1.0],
    "Mercurio": [0.78 * 2.5, 0.0, 0.0, 0, 1.65e-7],
    "Venus": [1.44 * 2.5, 0.0, 0.0, 0, 2.45e-6],
    "Tierra": [2 * 2.5, 0.0, 0.0, 0, 3.0e-6],
    "Marte": [3.04 * 2.5, 0.0, 0.0, 0, 3.2e-7],
    "Júpiter": [10.4 * 2.5, 0.0, 0.0, 0, 9.5e-4], 
    "Saturno": [19.16 * 2.5, 0.0, 0.0, 0, 2.86e-4],
    "Urano": [38.4 * 2.5, 0.0, 0.0, 0, 4.4e-5],
    "Neptuno": [60.1*2.5, 0.0, 0.0, 0, 5.15e-5],  
}
"""
planetas = {
    "Sol": [0.0 , 0.0, 0.0, 0.0, 1.0],
    "Mercurio": [2, 0.0, 0.0, 0, 1.65e-7],
    "Venus": [3, 0.0, 0.0, 0, 2.45e-6],
    "Tierra": [4, 0.0, 0.0, 0, 3.0e-6],
    "Marte": [5, 0.0, 0.0, 0, 3.2e-7],
    "Júpiter": [6, 0.0, 0.0, 0, 9.5e-4], 
    "Saturno": [7, 0.0, 0.0, 0, 2.86e-4],
    "Urano": [8, 0.0, 0.0, 0, 4.4e-5],
    "Neptuno": [9, 0.0, 0.0, 0, 5.15e-5],  
}


# Inicialización de velocidades para los planetas (excepto el Sol)
for p, data in planetas.items():
    if p != "Sol":
        x, y = data[0], data[1]
        r = np.sqrt(x**2 + y**2)
        data[2] = 0.0  # vx = 0
        data[3] = np.sqrt(G * Ms / r)  # vy = sqrt(GM/r)

# Imprimir datos iniciales de los planetas
def imprimir_datos_planetas(planetas):
    """
    Imprime por pantalla los datos iniciales de cada planeta: 
    posición (x, y), velocidad (vx, vy) y masa.
    """
    for nombre, datos in planetas.items():
        x, y, vx, vy, masa = datos
        print(f"Planeta: {nombre}")
        print(f"  Posición (x, y): ({x}, {y})")
        print(f"  Velocidad (vx, vy): ({vx}, {vy})")
        print(f"  Masa: {masa}\n")

imprimir_datos_planetas(planetas)

# Inicialización de arrays para almacenar posiciones y velocidades
posiciones = {planeta: [] for planeta in planetas}

# Función para calcular la aceleración gravitacional
def calcular_aceleracion(posiciones, masas):
    aceleraciones = {}
    for planeta1, (x1, y1) in posiciones.items():
        ax, ay = 0.0, 0.0
        for planeta2, (x2, y2) in posiciones.items():
            if planeta1 != planeta2:
                dx = x2 - x1
                dy = y2 - y1
                r = np.sqrt(dx**2 + dy**2)
                fuerza = G * masas[planeta2] / r**3
                ax += fuerza * dx
                ay += fuerza * dy
        aceleraciones[planeta1] = (ax, ay)
    return aceleraciones

# Inicialización de posiciones, velocidades y masas
pos_actual = {p: np.array([data[0], data[1]]) for p, data in planetas.items()}
vel_actual = {p: np.array([data[2], data[3]]) for p, data in planetas.items()}
masas = {p: data[4] for p, data in planetas.items()}

# Primer paso de Verlet (posición inicial y aceleración inicial)
aceleracion_actual = calcular_aceleracion(pos_actual, masas)

# Simulación con Verlet en velocidad
for step in range(num_steps):
    # Actualizar posiciones
    for planeta in planetas:
        pos_actual[planeta] += vel_actual[planeta] * dt + 0.5 * np.array(aceleracion_actual[planeta]) * dt**2

    # Calcular nuevas aceleraciones
    nueva_aceleracion = calcular_aceleracion(pos_actual, masas)

    # Actualizar velocidades
    for planeta in planetas:
        vel_actual[planeta] += 0.5 * (np.array(aceleracion_actual[planeta]) + np.array(nueva_aceleracion[planeta])) * dt

    # Guardar posiciones actuales
    for planeta in planetas:
        posiciones[planeta].append(pos_actual[planeta].tolist())

    # Actualizar aceleraciones
    aceleracion_actual = nueva_aceleracion

# Guardar resultados en un archivo .dat con el formato que la animación espera
with open(r"D:/DOCUMENTOS/GRANADA/4/COMPU/FisiCompis/planets_data.dat", "w") as f:
    for step in range(num_steps):
        # Un bloque (fotograma) por step => 1 línea por planeta
        for planeta in planetas:
            x, y = posiciones[planeta][step]
            # Escribe x,y separados por COMA
            f.write(f"{x},{y}\n")
        # Deja una línea en blanco para separar fotogramas
        f.write("\n")

print("Simulación completada. Las posiciones se han guardado en 'planets_data.dat'.")

Planeta: Sol
  Posición (x, y): (0.0, 0.0)
  Velocidad (vx, vy): (0.0, 0.0)
  Masa: 1.0

Planeta: Mercurio
  Posición (x, y): (2, 0.0)
  Velocidad (vx, vy): (0.0, 0.7071067811865476)
  Masa: 1.65e-07

Planeta: Venus
  Posición (x, y): (3, 0.0)
  Velocidad (vx, vy): (0.0, 0.5773502691896257)
  Masa: 2.45e-06

Planeta: Tierra
  Posición (x, y): (4, 0.0)
  Velocidad (vx, vy): (0.0, 0.5)
  Masa: 3e-06

Planeta: Marte
  Posición (x, y): (5, 0.0)
  Velocidad (vx, vy): (0.0, 0.4472135954999579)
  Masa: 3.2e-07

Planeta: Júpiter
  Posición (x, y): (6, 0.0)
  Velocidad (vx, vy): (0.0, 0.408248290463863)
  Masa: 0.00095

Planeta: Saturno
  Posición (x, y): (7, 0.0)
  Velocidad (vx, vy): (0.0, 0.3779644730092272)
  Masa: 0.000286

Planeta: Urano
  Posición (x, y): (8, 0.0)
  Velocidad (vx, vy): (0.0, 0.3535533905932738)
  Masa: 4.4e-05

Planeta: Neptuno
  Posición (x, y): (9, 0.0)
  Velocidad (vx, vy): (0.0, 0.3333333333333333)
  Masa: 5.15e-05

Simulación completada. Las posiciones se han guarda